<a href="https://colab.research.google.com/github/Tstrebe2/predicting-text-difficulty/blob/dave-updates/code/dave-eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

import nltk
import collections

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/696/processed.csv')

In [5]:
train_df = train_df[['original_text','processed_text','label']]
train_df['processed_text'] = train_df['processed_text'].astype('U')

,original_text,processed_text,label
0,There is manuscript evidence that Austen conti...,there is manuscript evidence that austen conti...,1
1,"In a remarkable comparative analysis , Mandaea...",in a remarkable comparative analysis mandaean...,1
2,"Before Persephone was released to Hermes , who...",before persephone was released to hermes who ...,1
3,Cogeneration plants are commonly found in dist...,cogeneration plants are commonly found in dist...,1
4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...",geneva is the second-most-populous ci...,1


In [6]:
easy_bigrams = collections.Counter()
diff_bigrams = collections.Counter()

In [7]:
train_df['token_text'] = train_df['processed_text'].apply(lambda x: [word for word in x.split() if word not in set(['','in','and','of','the','is','on','a'])] )
train_df.head()

,original_text,processed_text,label,token_text
0,There is manuscript evidence that Austen conti...,there is manuscript evidence that austen conti...,1,"[there, manuscript, evidence, that, austen, co..."
1,"In a remarkable comparative analysis , Mandaea...",in a remarkable comparative analysis mandaean...,1,"[remarkable, comparative, analysis, mandaean, ..."
2,"Before Persephone was released to Hermes , who...",before persephone was released to hermes who ...,1,"[before, persephone, was, released, to, hermes..."
3,Cogeneration plants are commonly found in dist...,cogeneration plants are commonly found in dist...,1,"[cogeneration, plants, are, commonly, found, d..."
4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...",geneva is the second-most-populous ci...,1,"[geneva, second-most-populous, city, switzerla..."


In [8]:
def CheckBiGrams(tokens,cntr):
    cntr.update(nltk.bigrams(tokens))

diff_train = train_df[train_df['label']==1]
easy_train = train_df[train_df['label']==0]

diff_train.apply(lambda x: CheckBiGrams(x['token_text'],diff_bigrams),axis=1)
easy_train.apply(lambda x: CheckBiGrams(x['token_text'],easy_bigrams),axis=1)

208384    None
208385    None
208386    None
208387    None
208388    None
          ... 
416763    None
416764    None
416765    None
416766    None
416767    None
Length: 208384, dtype: object

In [9]:
# 50 Most common bigrams for the difficult text
diff_bigrams.most_common(50)

[(('united', 'states'), 5610),
 (('known', 'as'), 5173),
 (('it', 'was'), 4222),
 (('to', 'be'), 3977),
 (('such', 'as'), 3766),
 (('he', 'was'), 3423),
 (('an', 'american'), 2521),
 (('as', 'well'), 2150),
 (('has', 'been'), 2128),
 (('can', 'be'), 2056),
 (('was', 'an'), 1935),
 (('well', 'as'), 1917),
 (('northern', 'france'), 1795),
 (('th', 'century'), 1789),
 (('also', 'known'), 1630),
 (('from', 'to'), 1630),
 (('new', 'york'), 1600),
 (('have', 'been'), 1587),
 (('was', 'born'), 1573),
 (('referred', 'to'), 1541),
 (('region', 'france'), 1460),
 (('for', 'his'), 1440),
 (('according', 'to'), 1409),
 (('football', 'player'), 1396),
 (('pas-de-calais', 'department'), 1384),
 (('commune', 'pas-de-calais'), 1382),
 (('there', 'are'), 1356),
 (('they', 'are'), 1354),
 (('as', 'an'), 1318),
 (('used', 'to'), 1309),
 (('due', 'to'), 1295),
 (('nord-pas-de-calais', 'region'), 1286),
 (('department', 'nord-pas-de-calais'), 1282),
 (('known', 'for'), 1282),
 (('to', 'as'), 1251),
 (('was

In [10]:
# 50 most common bigrams for easy to understand text
easy_bigrams.most_common(50)

[(('it', 'found'), 5362),
 (('found', 'region'), 5190),
 (('united', 'states'), 4985),
 (('it', 'was'), 4977),
 (('he', 'was'), 3915),
 (('football', 'player'), 3374),
 (('to', 'be'), 3019),
 (('known', 'as'), 2751),
 (('north', 'france'), 2320),
 (('department', 'north'), 2313),
 (('such', 'as'), 2295),
 (('can', 'be'), 2030),
 (('commune', 'it'), 1994),
 (('there', 'are'), 1847),
 (('an', 'american'), 1842),
 (('they', 'are'), 1622),
 (('was', 'an'), 1524),
 (('has', 'been'), 1496),
 (('was', 'born'), 1470),
 (('was', 'first'), 1393),
 (('new', 'york'), 1367),
 (('it', 'has'), 1334),
 (('used', 'to'), 1324),
 (('northwest', 'france'), 1284),
 (('department', 'northwest'), 1282),
 (('pas-de-calais', 'department'), 1240),
 (('th', 'century'), 1238),
 (('region', 'nord-pas-de-calais'), 1227),
 (('nord-pas-de-calais', 'pas-de-calais'), 1227),
 (('have', 'been'), 1145),
 (('from', 'to'), 1110),
 (('as', 'well'), 1095),
 (('de', 'la'), 1082),
 (('aisne', 'department'), 1075),
 (('region', 

In [14]:
# Evaluating whether there is a difference between mean token lengths between the easy/difficult texts
train_df.groupby('label')['token_text'].apply(lambda x: np.mean(x.str.len()))

label
0    11.765145
1    16.005403
Name: token_text, dtype: float64

In [21]:
# Evaluating what "features" are most important in our classification task...exploring what data we get back for tf-idf vectorizers

In [23]:
vectorizer = TfidfVectorizer(
                            min_df= 2,
                            max_df=.995,
                            max_features=5000,
                            ngram_range=(1,3),
                            stop_words='english')

In [24]:
X_train = vectorizer.fit_transform(train_df.processed_text)

In [26]:
y_train = train_df.label

In [28]:
import warnings
warnings.filterwarnings('ignore')
result = dict(zip(vectorizer.get_feature_names(), mutual_info_classif(X_train, y_train, discrete_features=True)))

In [30]:
sorted_result = sorted(result.items(), key=lambda item: item[1])

In [34]:
sorted_result[::-20]

[('born', 0.02809008544230052),
 ('national', 0.011030829235212475),
 ('war', 0.007541683652862186),
 ('early', 0.0065161366946063235),
 ('largest', 0.00581112677668762),
 ('long', 0.005122567273066937),
 ('professional', 0.004676612510432629),
 ('nord', 0.004343720239431437),
 ('song', 0.0040155686002334105),
 ('sea', 0.0036913653366483024),
 ('public', 0.003464518763428493),
 ('tropical', 0.0032784683638568952),
 ('european', 0.0030709656374351807),
 ('park', 0.00291862040149197),
 ('free', 0.002730889986587664),
 ('gironde', 0.0025995309802274626),
 ('having', 0.002487997763427144),
 ('christian', 0.0023080052381362582),
 ('lake', 0.0022365587815522036),
 ('department nord', 0.002146910979479698),
 ('little', 0.0020792413810416894),
 ('fourth', 0.0020225920416049267),
 ('region pays', 0.0019565916280076395),
 ('gave', 0.0019155089829945569),
 ('normandie region', 0.0018595158580498804),
 ('social', 0.0018054271721828425),
 ('surface', 0.0017690815445034803),
 ('spain', 0.00173638387

In [68]:
mnb = MultinomialNB().fit(X_train, y_train)

In [69]:
mnb.score(X_train, y_train)

0.6284215678746928

In [75]:
rf = RandomForestClassifier(n_estimators=100,
                       max_depth=100,
                       min_samples_split=10
                       ).fit(X_train,y_train)